# PART THREE

In [1]:
import sqlalchemy as sqla
import pymysql
import pandas as pd
import os

username = "user"  # get this from the instructor
password = "grad5100user"  # get this from the instructor
awsresource = "database-1.cwvjklnp4wu3.us-east-1.rds.amazonaws.com"  # get this from the instructor
port = 3306
dbname = "topChef"
dburi = f"mysql+pymysql://{username}:{password}@{awsresource}:{port}/{dbname}"
engine = sqla.create_engine(dburi)
def run_sql(sql, engine):
    "send sql to database given by engine and return the result"
    with engine.connect() as conn:
        result = conn.execute(sqla.text(sql))
    return result.all()


def df_select(sql, engine):
    "send sql to database given by engine and return the result as a pandas dataframe"
    with engine.connect() as conn:
        result = pd.read_sql(sqla.text(sql), con=conn)
    print(f"Retrieved {result.shape[0]} records")
    return result

In [2]:
tables = run_sql("show tables;", engine)
for x in tables:
    print(x)

('challengedescriptions',)
('challengewins',)
('chefdetails',)
('episodeinfo',)
('judges',)
('rewards',)


In [3]:
judges_info = run_sql("describe judges;", engine)
for x in judges_info[:10]:
    print(x)

('num', 'bigint', 'YES', 'MUL', None, '')
('season', 'text', 'YES', '', None, '')
('seasonNumber', 'bigint', 'YES', '', None, '')
('series', 'text', 'YES', '', None, '')
('episode', 'bigint', 'YES', '', None, '')
('challengeType', 'text', 'YES', '', None, '')
('outcomeType', 'text', 'YES', '', None, '')
('guestJudge', 'text', 'YES', '', None, '')
('competedOnTC', 'text', 'YES', '', None, '')
('otherShows', 'text', 'YES', '', None, '')


In [4]:
chefdetails_info = run_sql("describe chefdetails;", engine)
for x in chefdetails_info[:20]:
    print(x)

('num', 'bigint', 'YES', 'MUL', None, '')
('name', 'text', 'YES', '', None, '')
('chef', 'text', 'YES', '', None, '')
('hometown', 'text', 'YES', '', None, '')
('city', 'text', 'YES', '', None, '')
('state', 'text', 'YES', '', None, '')
('age', 'double', 'YES', '', None, '')
('season', 'text', 'YES', '', None, '')
('seasonNumber', 'bigint', 'YES', '', None, '')
('series', 'text', 'YES', '', None, '')
('placement', 'bigint', 'YES', '', None, '')
('personOfColor', 'text', 'YES', '', None, '')
('occupation', 'text', 'YES', '', None, '')
('gender', 'text', 'YES', '', None, '')


## 1. There are two types of top Chef programs; the ‘Masters’ programs and the regular series. The database contains information on both. We are not interested in the ‘Masters’. There is also a season called “Canada 6” which is part of the Masters series. Create versions of the chefdetails and judges tables that exclude any records coming from seasons that include the word Masters or Canada.

In [5]:
run_sql("select *FROM chefdetails WHERE season NOT LIKE '%Canada%' AND season NOT LIKE '%Master%';", engine)

[(0, 'Richard Blais', 'Richard B.', None, None, None, 38.0, 'All Stars: New York', 8, 'US', 1, None, 'Owner', 'Male'),
 (1, 'Mike Isabella', 'Mike I.', None, None, None, 35.0, 'All Stars: New York', 8, 'US', 2, None, 'Executive Chef/Owner', 'Male'),
 (2, 'Antonia Lofaso', 'Antonia L.', None, None, None, 34.0, 'All Stars: New York', 8, 'US', 3, None, 'Executive Chef', 'Female'),
 (3, 'Tiffany Derry', 'Tiffany D.', None, None, None, 27.0, 'All Stars: New York', 8, 'US', 4, 'POC', 'Executive Chef', 'Female'),
 (4, 'Carla Hall', 'Carla H.', None, None, None, 46.0, 'All Stars: New York', 8, 'US', 5, 'POC', 'Chef/Owner', 'Female'),
 (5, 'Dale Talde', 'Dale T.', None, None, None, 32.0, 'All Stars: New York', 8, 'US', 6, 'POC', 'Chef de Cuisine', 'Male'),
 (6, 'Angelo Sosa', 'Angelo S.', None, None, None, 35.0, 'All Stars: New York', 8, 'US', 7, 'POC', 'President', 'Male'),
 (7, 'Fabio Viviani', 'Fabio V.', None, None, None, 32.0, 'All Stars: New York', 8, 'US', 8, None, 'Owner', 'Male'),
 (8,

In [6]:
run_sql("select *FROM judges WHERE season NOT LIKE '%Canada%' AND season NOT LIKE '%Master%';", engine)

[(0, 'All Stars: New York', 8, 'US', 1, 'Quickfire', 'Team', 'Tom Colicchio', None, 'Top Chef Judge'),
 (1, 'All Stars: New York', 8, 'US', 1, 'Elimination', 'Individual', 'Anthony Bourdain', None, 'Parts Unknown; No Reservations'),
 (2, 'All Stars: New York', 8, 'US', 2, 'Quickfire', 'Individual', 'Joe Jonas', None, None),
 (3, 'All Stars: New York', 8, 'US', 2, 'Elimination', 'Team', 'Katie Lee', None, 'Beat Bobby Flay Judge'),
 (4, 'All Stars: New York', 8, 'US', 3, 'Quickfire', 'Team', 'David Chang', None, None),
 (5, 'All Stars: New York', 8, 'US', 3, 'Elimination', 'Team', 'Anthony Bourdain', None, 'Parts Unknown; No Reservations'),
 (6, 'All Stars: New York', 8, 'US', 3, 'Elimination', 'Team', 'Kate Krader', None, None),
 (7, 'All Stars: New York', 8, 'US', 4, 'Quickfire', 'Individual', 'Tony Mantuano', None, 'Top Chef Masters Season 2'),
 (8, 'All Stars: New York', 8, 'US', 4, 'Elimination', 'Team', 'Tony Mantuano', None, 'Top Chef Masters Season 2'),
 (9, 'All Stars: New York'

We can return the sql table as a dataframe

In [7]:
with engine.connect() as conn:
    chefdetails_1 = pd.read_sql(
        sqla.text("select *FROM chefdetails WHERE season NOT LIKE '%Canada%' AND season NOT LIKE '%Master%';"),
        conn,
    )
chefdetails_1.head()

,num,name,chef,hometown,city,state,age,season,seasonNumber,series,placement,personOfColor,occupation,gender
0,0,Richard Blais,Richard B.,None,None,None,38.0,All Stars: New York,8,US,1,None,Owner,Male
1,1,Mike Isabella,Mike I.,None,None,None,35.0,All Stars: New York,8,US,2,None,Executive Chef/Owner,Male
2,2,Antonia Lofaso,Antonia L.,None,None,None,34.0,All Stars: New York,8,US,3,None,Executive Chef,Female
3,3,Tiffany Derry,Tiffany D.,None,None,None,27.0,All Stars: New York,8,US,4,POC,Executive Chef,Female
4,4,Carla Hall,Carla H.,None,None,None,46.0,All Stars: New York,8,US,5,POC,Chef/Owner,Female


In [8]:
with engine.connect() as conn:
    judges_1 = pd.read_sql(
        sqla.text("select *FROM judges WHERE season NOT LIKE '%Canada%' AND season NOT LIKE '%Master%';"),
        conn,
    )
judges_1.head()

,num,season,seasonNumber,series,episode,challengeType,outcomeType,guestJudge,competedOnTC,otherShows
0,0,All Stars: New York,8,US,1,Quickfire,Team,Tom Colicchio,None,Top Chef Judge
1,1,All Stars: New York,8,US,1,Elimination,Individual,Anthony Bourdain,None,Parts Unknown; No Reservations
2,2,All Stars: New York,8,US,2,Quickfire,Individual,Joe Jonas,None,None
3,3,All Stars: New York,8,US,2,Elimination,Team,Katie Lee,None,Beat Bobby Flay Judge
4,4,All Stars: New York,8,US,3,Quickfire,Team,David Chang,None,None


## 2. Further trim your chefdetails and judges tables by including only the columns of interest from the table descriptions above.

In [13]:
run_sql("select season, seasonNumber, name, placement, gender" 
        " FROM chefdetails WHERE season NOT LIKE '%Canada%' AND season NOT LIKE '%Master%';", engine)

[('All Stars: New York', 8, 'Richard Blais', 1, 'Male'),
 ('All Stars: New York', 8, 'Mike Isabella', 2, 'Male'),
 ('All Stars: New York', 8, 'Antonia Lofaso', 3, 'Female'),
 ('All Stars: New York', 8, 'Tiffany Derry', 4, 'Female'),
 ('All Stars: New York', 8, 'Carla Hall', 5, 'Female'),
 ('All Stars: New York', 8, 'Dale Talde', 6, 'Male'),
 ('All Stars: New York', 8, 'Angelo Sosa', 7, 'Male'),
 ('All Stars: New York', 8, 'Fabio Viviani', 8, 'Male'),
 ('All Stars: New York', 8, 'Tre Wilcox', 9, 'Male'),
 ('All Stars: New York', 8, 'Marcel Vigneron', 10, 'Male'),
 ('All Stars: New York', 8, 'Jamie Lauren', 11, 'Female'),
 ('All Stars: New York', 8, 'Tiffani Faison', 11, 'Female'),
 ('All Stars: New York', 8, 'Casey Thompson', 13, 'Female'),
 ('All Stars: New York', 8, 'Spike Mendelsohn', 14, 'Male'),
 ('All Stars: New York', 8, 'Dale Levitski', 15, 'Male'),
 ('All Stars: New York', 8, 'Stephen Asprinio', 15, 'Male'),
 ('All Stars: New York', 8, 'Jennifer Carroll', 17, 'Female'),
 ('All 

In [14]:
run_sql("select season, seasonNumber, episode, challengeType, guestJudge, competedOnTC" 
        " FROM judges WHERE season NOT LIKE '%Canada%' AND season NOT LIKE '%Master%';", engine)

[('All Stars: New York', 8, 1, 'Quickfire', 'Tom Colicchio', None),
 ('All Stars: New York', 8, 1, 'Elimination', 'Anthony Bourdain', None),
 ('All Stars: New York', 8, 2, 'Quickfire', 'Joe Jonas', None),
 ('All Stars: New York', 8, 2, 'Elimination', 'Katie Lee', None),
 ('All Stars: New York', 8, 3, 'Quickfire', 'David Chang', None),
 ('All Stars: New York', 8, 3, 'Elimination', 'Anthony Bourdain', None),
 ('All Stars: New York', 8, 3, 'Elimination', 'Kate Krader', None),
 ('All Stars: New York', 8, 4, 'Quickfire', 'Tony Mantuano', None),
 ('All Stars: New York', 8, 4, 'Elimination', 'Tony Mantuano', None),
 ('All Stars: New York', 8, 5, 'Quickfire', 'Tom Colicchio', None),
 ('All Stars: New York', 8, 5, 'Elimination', 'Susur Lee', None),
 ('All Stars: New York', 8, 6, 'Elimination', 'Kerry Heffernan', None),
 ('All Stars: New York', 8, 7, 'Quickfire', 'Justo Thomas', None),
 ('All Stars: New York', 8, 7, 'Elimination', 'Ludo Lefebvre', None),
 ('All Stars: New York', 8, 8, 'Quickfire

We can also return a sql table as dataframe.

In [14]:
columns_of_interest = ['season', 'seasonNumber', 'name', 'placement', 'gender']
chefdetails_2 = chefdetails_1[columns_of_interest]
chefdetails_2.head()

,season,seasonNumber,name,placement,gender
0,All Stars: New York,8,Richard Blais,1,Male
1,All Stars: New York,8,Mike Isabella,2,Male
2,All Stars: New York,8,Antonia Lofaso,3,Female
3,All Stars: New York,8,Tiffany Derry,4,Female
4,All Stars: New York,8,Carla Hall,5,Female


In [15]:
columns_of_interest = ['season', 'seasonNumber', 'episode', 'challengeType', 'guestJudge', 'competedOnTC']
judges_2 = judges_1[columns_of_interest]
judges_2.head()

,season,seasonNumber,episode,challengeType,guestJudge,competedOnTC
0,All Stars: New York,8,1,Quickfire,Tom Colicchio,None
1,All Stars: New York,8,1,Elimination,Anthony Bourdain,None
2,All Stars: New York,8,2,Quickfire,Joe Jonas,None
3,All Stars: New York,8,2,Elimination,Katie Lee,None
4,All Stars: New York,8,3,Quickfire,David Chang,None


## 3. Your tables should have information from 20 remaining different seasons, numbered from 1 to 20. Make a table that has two columns, one called season and one called seasonNumber showing the number associated with each season. (For example, San Francisco is season 1.)

In [31]:
run_sql("select DISTINCT season, seasonNumber" 
        " FROM chefdetails WHERE season NOT LIKE '%Canada%' AND season NOT LIKE '%Master%';", engine)

[('All Stars: New York', 8),
 ('All-Stars L.A.', 17),
 ('Boston', 12),
 ('California', 13),
 ('Charleston', 14),
 ('Chicago', 4),
 ('D.C.', 7),
 ('Colorado', 15),
 ('Houston', 19),
 ('Kentucky', 16),
 ('Las Vegas', 6),
 ('Los Angeles', 2),
 ('Miami', 3),
 ('New Orleans', 11),
 ('New York', 5),
 ('Portland', 18),
 ('San Francisco', 1),
 ('Seattle', 10),
 ('Texas', 9),
 ('World All Stars', 20)]

We can also get this by working on dataframe.

In [16]:
chefdetails_3 = chefdetails_2[['season', 'seasonNumber']].drop_duplicates().reset_index(drop=True)
chefdetails_3

,season,seasonNumber
0,All Stars: New York,8
1,All-Stars L.A.,17
2,Boston,12
3,California,13
4,Charleston,14
5,Chicago,4
6,D.C.,7
7,Colorado,15
8,Houston,19
9,Kentucky,16


## 4. Answer the following questions using your tables so far.

### a. Among all chef contestants, how many are male and how many are female?

I will work on the table got by question 1.

In [51]:
run_sql("select COUNT(DISTINCT name) FROM chefdetails "
        "WHERE season NOT LIKE '%Canada%' AND season NOT LIKE '%Master%' AND gender = 'Male';", engine)

[(144,)]

In [52]:
run_sql("select COUNT(DISTINCT name) FROM chefdetails "
        "WHERE season NOT LIKE '%Canada%' AND season NOT LIKE '%Master%' AND gender = 'Female';", engine)

[(128,)]

I can also get the anwser by working on dataframe.  
First I remove duplicate names from all chef contestants, and then calculate the number of male and female separately.

In [17]:
chefdetails_4_a = chefdetails_1[['name', 'gender',]].drop_duplicates()

In [18]:
chefdetails_male = chefdetails_4_a[chefdetails_4_a['gender'] == 'Male']
len(chefdetails_male)

144

In [19]:
chefdetails_female = chefdetails_4_a[chefdetails_4_a['gender'] == 'Female']
len(chefdetails_female)

128

So Among all chef contestants, 144 are male and 128 are female.

### b. Among all winners (placement==1) how many are male and how many are female?

In [71]:
run_sql("select COUNT(name) FROM chefdetails "
        "WHERE gender = 'Male' AND placement = 1 AND season NOT LIKE '%Canada%' AND season NOT LIKE '%Master%';", engine)

[(14,)]

In [72]:
run_sql("select COUNT(name) FROM chefdetails "
        "WHERE gender = 'Female' AND placement = 1 AND season NOT LIKE '%Canada%' AND season NOT LIKE '%Master%';", engine)

[(6,)]

I can also get the anwser by working on dataframe.

In [73]:
chefdetails_4_b = chefdetails_1[chefdetails_1['placement'] == 1]
chefdetails_4_b = chefdetails_4_b[['name', 'gender']]

In [74]:
chefdetails_male = chefdetails_4_b[chefdetails_4_b['gender'] == 'Male']
len(chefdetails_male)

14

In [75]:
chefdetails_female = chefdetails_4_b[chefdetails_4_b['gender'] == 'Female']
len(chefdetails_female)

6

So Among all winners (placement==1) 13 are male and 6 are female.

### c. Among all sets of top 3 finalists (placement=1,2,3) how many are male and how many are female?

In [76]:
run_sql("select COUNT(name) FROM chefdetails "
        "WHERE gender = 'Male' AND (placement = 1 OR placement = 2 OR placement = 3) "
        "AND season NOT LIKE '%Canada%' AND season NOT LIKE '%Master%';", engine)       

[(36,)]

In [77]:
run_sql("select COUNT(name) FROM chefdetails "
        "WHERE gender = 'Female' AND (placement = 1 OR placement = 2 OR placement = 3) " 
        "AND season NOT LIKE '%Canada%' AND season NOT LIKE '%Master%';", engine) 

[(25,)]

We can also get the answer by working on dataframe.

In [78]:
chefdetails_4_c = chefdetails_1[chefdetails_1['placement'] <= 3]

In [79]:
chefdetails_4_c = chefdetails_4_c[['name', 'gender']]

In [80]:
chefdetails_male = chefdetails_4_c[chefdetails_4_c['gender'] == 'Male']
len(chefdetails_male)

36

In [81]:
chefdetails_female = chefdetails_4_c[chefdetails_4_c['gender'] == 'Female']
len(chefdetails_female)

25

So among all sets of top 3 finalists (placement=1,2,3) 32 are male and 22 are female.

### d. Compare the number of times a female was placed in the top 3 with the number of times a female won. Does this number seem unusual? That is, are women who place in the top 3 less likely than expected to ultimately win the competition? Why or why not?

Yes, women who place in the top 3 are less likely than expected to ultimately win the competition. Since we expect that women get one win for every three places in the top 3 on average. So according to 25 times top 3, they should win for 25/3 = 8 times, but there are 6 times in reality. This may be because the random error, the amount of sample is too small. If the sample is large enough, I think the ratio will be close to 1/3. Waht's more, more than one contestants can get the same rank at the same season, which is another reason for the unusual number.

## 5.  Make a table with one row for each first place winner containing the winner’s name, the season they won, and a column containing ‘Yes’ or ‘No’ depending on whether they served as a guest judge or not.

In [11]:
run_sql("select DISTINCT name, chefdetails.season, COALESCE(competedOnTC, 'No') AS competedOnTC "
        "FROM chefdetails LEFT JOIN judges ON chefdetails.name = judges.guestJudge WHERE chefdetails.placement = 1 AND chefdetails.season NOT LIKE '%Canada%' AND chefdetails.season NOT LIKE '%Master%';", engine) 

[('Richard Blais', 'All Stars: New York', 'Yes'),
 ('Melissa King', 'All-Stars L.A.', 'Yes'),
 ('Mei Lin', 'Boston', 'No'),
 ('Jeremy Ford', 'California', 'No'),
 ('Brooke Williamson', 'Charleston', 'Yes'),
 ('Stephanie Izard', 'Chicago', 'Yes'),
 ('Kevin Sbraga', 'D.C.', 'No'),
 ('Joseph Flamm', 'Colorado', 'No'),
 ('Kah-wai Lo', 'Houston', 'No'),
 ('Kelsey Barnard Clark', 'Kentucky', 'Yes'),
 ('Michael Voltaggio', 'Las Vegas', 'Yes'),
 ('Ilan Hall', 'Los Angeles', 'Yes'),
 ('Hung Huynh', 'Miami', 'Yes'),
 ('Nicholas Elmi', 'New Orleans', 'No'),
 ('Hosea Rosenberg', 'New York', 'Yes'),
 ('Gabe Erales', 'Portland', 'No'),
 ('Harold Dieterle', 'San Francisco', 'Yes'),
 ('Kristen Kish', 'Seattle', 'Yes'),
 ('Paul Qui', 'Texas', 'No'),
 ('Kah-wai Lo', 'World All Stars', 'No')]

We can also get the result by working on dataframe.

In [27]:
chefdetails_5 = chefdetails_1[chefdetails_1['placement'] == 1]
chefdetails_5 = chefdetails_5[['name', 'season']]

In [28]:
judges_5 = judges_1[['guestJudge', 'competedOnTC']]

In [29]:
chefdetails_5_merge = pd.merge(chefdetails_5, judges_5, left_on='name', right_on='guestJudge', 
                               how='left').drop_duplicates(subset=['name', 'season'])

In [30]:
chefdetails_5_new = chefdetails_5_merge.drop('guestJudge', axis=1, inplace=False)

In [31]:
chefdetails_5_new.fillna('No', inplace=True)

In [32]:
chefdetails_5_new

,name,season,competedOnTC
0,Richard Blais,All Stars: New York,Yes
7,Melissa King,All-Stars L.A.,Yes
10,Mei Lin,Boston,No
11,Jeremy Ford,California,No
12,Brooke Williamson,Charleston,Yes
20,Stephanie Izard,Chicago,Yes
25,Kevin Sbraga,D.C.,No
26,Joseph Flamm,Colorado,No
27,Kah-wai Lo,Houston,No
28,Kelsey Barnard Clark,Kentucky,Yes


### 6. Find all contestants who were not first place winners but did serve as a guest judge at some point. Make sure to only list each name once.

In [33]:
chefdetails_6 = chefdetails_1[chefdetails_1['placement'] != 1]
chefdetails_6 = chefdetails_6[['name']]

In [35]:
chefdetails_6_merge = pd.merge(chefdetails_6, judges_5, left_on='name', right_on='guestJudge', 
                               how='left').drop_duplicates(subset=['name'])

In [46]:
chefdetails_6_merge = chefdetails_6_merge[chefdetails_6_merge['competedOnTC']=='Yes']
chefdetails_6_merge = chefdetails_6_merge[['name']]

In [55]:
chefdetails_6_1 = pd.merge(chefdetails_6_merge, chefdetails_5_new, on='name', how='left')
chefdetails_6_1 = chefdetails_6_1[chefdetails_6_1['competedOnTC'] == 'Yes']

In [56]:
chefdetails_6_new = chefdetails_6_1['name']

In [67]:
chefdetails_6_new = chefdetails_6_merge[chefdetails_6_merge['name'] != 'Melissa King']
chefdetails_6_new = chefdetails_6_new[chefdetails_6_new['name'] != 'Richard Blais']
chefdetails_6_new = chefdetails_6_new[chefdetails_6_new['name'] != 'Brooke Williamson']

All contestants who were not first place winners but did serve as a guest judge at some point are listed on the following.

In [70]:
chefdetails_6_new

,name
0,Mike Isabella
1,Antonia Lofaso
2,Tiffany Derry
5,Dale Talde
13,Spike Mendelsohn
15,Stephen Asprinio
18,Bryan Voltaggio
24,Gregory Gourdet
26,Karen Akunowicz
27,Lee Anne Wong
